In [1]:
param_x_api_key = '2BeBXg69E53RaoUFhA7hhf17jCg1NkwHoJz3FxmJPNS3bedbpHcb'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_part = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != 0:
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_part[input_.loc[i, 'JOIN_PNBI']] = {'vehicle_page': vehicle_page,
                                                 'crawler': 0,
                                                 'dict_vehicle': {},
                                                 'vehicle': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                data = '''{"getAutoCarePartApplications":
                {"brandCode":"''' + input_.loc[i, 'Brand_Id'] + '''",
                 "partNumber":"''' + input_.loc[i, 'Part_Number'] + '''",
                 "includeResults":true,
                 "includeMatchingBaseVehicles":true,
                 "perPage":100,
                 "page":''' + str(j+1) + '''}}'''

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.opticatonline.com/autocare/v1/services/Catalog.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                for result in json_data['results']:
                    make = result['matchingBaseVehicles'][0]['makeName'].strip()
                    model = result['matchingBaseVehicles'][0]['modelName'].strip()
                    year = result['matchingBaseVehicles'][0]['year']

                    if make not in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle']:
                        dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make] = {model: [year]}
                    else:
                        if model not in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make]:
                            dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model] = [year]
                        else:
                            dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model].append(year)

                    dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model] = sorted(list(set(dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model])))

                dict_part[input_.loc[i, 'JOIN_PNBI']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_part[input_.loc[i, 'JOIN_PNBI']]['vehicle_page'] == dict_part[input_.loc[i, 'JOIN_PNBI']]['crawler']:
                    list_vehicle = []
                    for make in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle']:
                        for model in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make]:
                            year_min = min(dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model])
                            year_max = max(dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model])

                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    list_vehicle.sort()

                    dict_part[input_.loc[i, 'JOIN_PNBI']]['vehicle'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'JOIN_PNBI': input_.loc[i, 'JOIN_PNBI'],
                                             'Vehicle': dict_part[input_.loc[i, 'JOIN_PNBI']]['vehicle']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[i, 'Part_Number'],
                                     'Brand_Id': input_.loc[i, 'Brand_Id'],
                                     'JOIN_PNBI': input_.loc[i, 'JOIN_PNBI']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'JOIN_PNBI'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_PNBI'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number', 'Brand_Id'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1733

[ok] - MF0385;GGNF
[尝试次数：1] - [剩余数量：1713] - [当前时间：20:42:50]

[ok] - MF0413;GGNF
[尝试次数：1] - [剩余数量：1712] - [当前时间：20:42:51]

[ok] - MF20063;GGNF
[尝试次数：1] - [剩余数量：1711] - [当前时间：20:42:51]

[ok] - MF20046;GGNF
[尝试次数：1] - [剩余数量：1710] - [当前时间：20:42:51]

[ok] - MF0419;GGNF
[尝试次数：1] - [剩余数量：1709] - [当前时间：20:42:51]

[ok] - MF20071;GGNF
[尝试次数：1] - [剩余数量：1708] - [当前时间：20:42:51]

[ok] - MF0422;GGNF
[尝试次数：1] - [剩余数量：1707] - [当前时间：20:42:51]

[ok] - MF20060;GGNF
[尝试次数：1] - [剩余数量：1706] - [当前时间：20:42:51]

[ok] - MF0917;GGNF
[尝试次数：1] - [剩余数量：1705] - [当前时间：20:42:52]

[ok] - MF20082;GGNF
[尝试次数：1] - [剩余数量：1704] - [当前时间：20:42:52]

[ok] - MF21183;GGNF
[尝试次数：1] - [剩余数量：1703] - [当前时间：20:42:52]

[ok] - MF21178;GGNF
[尝试次数：1] - [剩余数量：1702] - [当前时间：20:42:52]

[ok] - MF21185;GGNF
[尝试次数：1] - [剩余数量：1701] - [当前时间：20:42:52]

[ok] - MF21205;GGNF
[尝试次数：1] - [剩余数量：1700] - [当前时间：20:42:52]

[ok] - MF20066;GGNF
[尝试次数：1] - [剩余数量：1699] - [当前时间：20:42:52]

[ok] - MF20073;GGNF
[尝试次数：1] - [剩余数量：1698] - [当前时间：20:42:53]

[ok

[ok] - MF20090;GGNF
[尝试次数：1] - [剩余数量：1579] - [当前时间：20:43:15]

[ok] - MF20050;GGNF
[尝试次数：1] - [剩余数量：1578] - [当前时间：20:43:15]

[ok] - MF20052;GGNF
[尝试次数：1] - [剩余数量：1577] - [当前时间：20:43:15]

[ok] - MF20127;GGNF
[尝试次数：1] - [剩余数量：1576] - [当前时间：20:43:15]

[ok] - MF21006;GGNF
[尝试次数：1] - [剩余数量：1575] - [当前时间：20:43:16]

[ok] - MF21005;GGNF
[尝试次数：1] - [剩余数量：1574] - [当前时间：20:43:16]

[ok] - MF21008;GGNF
[尝试次数：1] - [剩余数量：1573] - [当前时间：20:43:16]

[ok] - MF21009;GGNF
[尝试次数：1] - [剩余数量：1572] - [当前时间：20:43:17]

[ok] - MF21011;GGNF
[尝试次数：1] - [剩余数量：1571] - [当前时间：20:43:17]

[ok] - MF21014;GGNF
[尝试次数：1] - [剩余数量：1570] - [当前时间：20:43:17]

[ok] - MF21013;GGNF
[尝试次数：1] - [剩余数量：1569] - [当前时间：20:43:18]

[ok] - MF21016;GGNF
[尝试次数：1] - [剩余数量：1568] - [当前时间：20:43:19]

[ok] - MF21023;GGNF
[尝试次数：1] - [剩余数量：1567] - [当前时间：20:43:19]

[ok] - MF21024;GGNF
[尝试次数：1] - [剩余数量：1566] - [当前时间：20:43:19]

[ok] - MF21027;GGNF
[尝试次数：1] - [剩余数量：1565] - [当前时间：20:43:20]

[ok] - MF20069;GGNF
[尝试次数：1] - [剩余数量：1564] - [当前时间：20:43:21]

[ok] - M

[ok] - MF21186;GGNF
[尝试次数：1] - [剩余数量：1445] - [当前时间：20:44:25]

[ok] - MF21189;GGNF
[尝试次数：1] - [剩余数量：1444] - [当前时间：20:44:25]

[ok] - MF21190;GGNF
[尝试次数：1] - [剩余数量：1443] - [当前时间：20:44:25]

[ok] - MF21191;GGNF
[尝试次数：1] - [剩余数量：1442] - [当前时间：20:44:25]

[ok] - MF21192;GGNF
[尝试次数：1] - [剩余数量：1441] - [当前时间：20:44:26]

[ok] - MF21192;GGNF
[尝试次数：1] - [剩余数量：1440] - [当前时间：20:44:26]

[ok] - MF21192;GGNF
[尝试次数：1] - [剩余数量：1439] - [当前时间：20:44:26]

[ok] - MF20042;GGNF
[尝试次数：2] - [剩余数量：1438] - [当前时间：20:44:26]

[ok] - MF21194;GGNF
[尝试次数：1] - [剩余数量：1437] - [当前时间：20:44:26]

[ok] - MF21195;GGNF
[尝试次数：1] - [剩余数量：1436] - [当前时间：20:44:27]

[ok] - MF21197;GGNF
[尝试次数：1] - [剩余数量：1435] - [当前时间：20:44:27]

[ok] - MF21197;GGNF
[尝试次数：1] - [剩余数量：1434] - [当前时间：20:44:27]

[ok] - MF21181;GGNF
[尝试次数：1] - [剩余数量：1433] - [当前时间：20:44:27]

[ok] - MF20080;GGNF
[尝试次数：5] - [剩余数量：1432] - [当前时间：20:44:28]

[ok] - MF21198;GGNF
[尝试次数：1] - [剩余数量：1431] - [当前时间：20:44:28]

[ok] - MF21196;GGNF
[尝试次数：1] - [剩余数量：1430] - [当前时间：20:44:28]

[ok] - M

[ok] - MF3940;GGNF
[尝试次数：4] - [剩余数量：1312] - [当前时间：20:44:49]

[ok] - MF21073;GGNF
[尝试次数：2] - [剩余数量：1311] - [当前时间：20:44:49]

[ok] - MF21396;GGNF
[尝试次数：4] - [剩余数量：1310] - [当前时间：20:44:50]

[ok] - MF4219;GGNF
[尝试次数：3] - [剩余数量：1309] - [当前时间：20:44:50]

[ok] - MF4220;GGNF
[尝试次数：2] - [剩余数量：1308] - [当前时间：20:44:50]

[ok] - MF3985;GGNF
[尝试次数：7] - [剩余数量：1307] - [当前时间：20:44:50]

[ok] - MF4215;GGNF
[尝试次数：1] - [剩余数量：1306] - [当前时间：20:44:50]

[ok] - MF4227;GGNF
[尝试次数：2] - [剩余数量：1305] - [当前时间：20:44:51]

[ok] - MF4230;GGNF
[尝试次数：1] - [剩余数量：1304] - [当前时间：20:44:51]

[ok] - MF21386;GGNF
[尝试次数：7] - [剩余数量：1303] - [当前时间：20:44:51]

[ok] - MF21396;GGNF
[尝试次数：2] - [剩余数量：1302] - [当前时间：20:44:51]

[ok] - MF5519;GGNF
[尝试次数：1] - [剩余数量：1301] - [当前时间：20:44:52]

[ok] - MF4235;GGNF
[尝试次数：5] - [剩余数量：1300] - [当前时间：20:44:53]

[ok] - MF4202;GGNF
[尝试次数：5] - [剩余数量：1299] - [当前时间：20:44:53]

[ok] - MF4224;GGNF
[尝试次数：3] - [剩余数量：1298] - [当前时间：20:44:53]

[ok] - MF7557;GGNF
[尝试次数：1] - [剩余数量：1297] - [当前时间：20:44:54]

[ok] - MF6309;GGNF
[

[ok] - MF21359;GGNF
[尝试次数：1] - [剩余数量：1177] - [当前时间：20:45:25]

[ok] - MF21364;GGNF
[尝试次数：1] - [剩余数量：1176] - [当前时间：20:45:25]

[ok] - MF21364;GGNF
[尝试次数：1] - [剩余数量：1175] - [当前时间：20:45:25]

[ok] - MF21359;GGNF
[尝试次数：1] - [剩余数量：1174] - [当前时间：20:45:25]

[ok] - MF2967;GGNF
[尝试次数：1] - [剩余数量：1173] - [当前时间：20:45:26]

[ok] - MF2966;GGNF
[尝试次数：1] - [剩余数量：1172] - [当前时间：20:45:26]

[ok] - MF21364;GGNF
[尝试次数：1] - [剩余数量：1171] - [当前时间：20:45:26]

[ok] - MF4234;GGNF
[尝试次数：1] - [剩余数量：1170] - [当前时间：20:45:27]

[ok] - MF50014A;GGNF
[尝试次数：1] - [剩余数量：1169] - [当前时间：20:45:27]

[ok] - MF5288;GGNF
[尝试次数：1] - [剩余数量：1168] - [当前时间：20:45:27]

[ok] - MF8302;GGNF
[尝试次数：1] - [剩余数量：1167] - [当前时间：20:45:27]

[ok] - MF8302;GGNF
[尝试次数：1] - [剩余数量：1166] - [当前时间：20:45:27]

[ok] - MF8302;GGNF
[尝试次数：1] - [剩余数量：1165] - [当前时间：20:45:27]

[ok] - MF21364;GGNF
[尝试次数：1] - [剩余数量：1164] - [当前时间：20:45:27]

[ok] - MF8302;GGNF
[尝试次数：1] - [剩余数量：1163] - [当前时间：20:45:28]

[ok] - MF5288;GGNF
[尝试次数：1] - [剩余数量：1162] - [当前时间：20:45:28]

[ok] - MF8302;GG

...

[ok] - 59072;GHRL
[尝试次数：1] - [剩余数量：623] - [当前时间：20:48:25]

[ok] - 59064;GHRL
[尝试次数：2] - [剩余数量：622] - [当前时间：20:48:26]

[ok] - 59072;GHRL
[尝试次数：2] - [剩余数量：621] - [当前时间：20:48:26]

[ok] - 59052;GHRL
[尝试次数：2] - [剩余数量：620] - [当前时间：20:48:26]

[ok] - 59085;GHRL
[尝试次数：1] - [剩余数量：619] - [当前时间：20:48:27]

[ok] - 59085;GHRL
[尝试次数：1] - [剩余数量：618] - [当前时间：20:48:27]

[ok] - 59068;GHRL
[尝试次数：7] - [剩余数量：617] - [当前时间：20:48:27]

[ok] - 59085;GHRL
[尝试次数：3] - [剩余数量：616] - [当前时间：20:48:27]

[ok] - 59085;GHRL
[尝试次数：2] - [剩余数量：615] - [当前时间：20:48:28]

[ok] - 59085;GHRL
[尝试次数：1] - [剩余数量：614] - [当前时间：20:48:28]

[ok] - 59085;GHRL
[尝试次数：6] - [剩余数量：613] - [当前时间：20:48:28]

[ok] - 59085;GHRL
[尝试次数：1] - [剩余数量：612] - [当前时间：20:48:28]

[ok] - 59162;GHRL
[尝试次数：2] - [剩余数量：611] - [当前时间：20:48:29]

[ok] - 59085;GHRL
[尝试次数：1] - [剩余数量：610] - [当前时间：20:48:29]

[ok] - 59085;GHRL
[尝试次数：3] - [剩余数量：609] - [当前时间：20:48:29]

[ok] - 59086;GHRL
[尝试次数：1] - [剩余数量：608] - [当前时间：20:48:29]

[ok] - 59085;GHRL
[尝试次数：3] - [剩余数量：607] - [当前时间：20:48:29

[ok] - 221203;GGNG
[尝试次数：3] - [剩余数量：485] - [当前时间：20:48:57]

[ok] - 221224;GGNG
[尝试次数：2] - [剩余数量：484] - [当前时间：20:48:58]

[ok] - 221258;GGNG
[尝试次数：1] - [剩余数量：483] - [当前时间：20:48:58]

[ok] - 221258;GGNG
[尝试次数：1] - [剩余数量：482] - [当前时间：20:48:59]

[ok] - 221203;GGNG
[尝试次数：1] - [剩余数量：481] - [当前时间：20:48:59]

[ok] - 20427;GGNG
[尝试次数：2] - [剩余数量：480] - [当前时间：20:48:59]

[ok] - 221359;GGNG
[尝试次数：1] - [剩余数量：479] - [当前时间：20:49:00]

[ok] - 221359;GGNG
[尝试次数：1] - [剩余数量：478] - [当前时间：20:49:00]

[ok] - 221224;GGNG
[尝试次数：1] - [剩余数量：477] - [当前时间：20:49:00]

[ok] - 221258;GGNG
[尝试次数：1] - [剩余数量：476] - [当前时间：20:49:00]

[ok] - 221359;GGNG
[尝试次数：1] - [剩余数量：475] - [当前时间：20:49:01]

[ok] - 221359;GGNG
[尝试次数：1] - [剩余数量：474] - [当前时间：20:49:01]

[ok] - 221359;GGNG
[尝试次数：1] - [剩余数量：473] - [当前时间：20:49:01]

[ok] - 221359;GGNG
[尝试次数：1] - [剩余数量：472] - [当前时间：20:49:01]

[ok] - 221258;GGNG
[尝试次数：1] - [剩余数量：471] - [当前时间：20:49:02]

[ok] - 26313M;GGNG
[尝试次数：1] - [剩余数量：470] - [当前时间：20:49:02]

[ok] - 250014A;GGNG
[尝试次数：1] - [剩余数量：469]

[ok] - 621088F;GHRL
[尝试次数：1] - [剩余数量：346] - [当前时间：20:49:39]

[ok] - 621120F;GHRL
[尝试次数：1] - [剩余数量：345] - [当前时间：20:49:39]

[ok] - 621136F;GHRL
[尝试次数：1] - [剩余数量：344] - [当前时间：20:49:39]

[ok] - 621058F;GHRL
[尝试次数：1] - [剩余数量：343] - [当前时间：20:49:40]

[ok] - 621142F;GHRL
[尝试次数：1] - [剩余数量：342] - [当前时间：20:49:40]

[ok] - 621066F;GHRL
[尝试次数：1] - [剩余数量：341] - [当前时间：20:49:40]

[ok] - 621150F;GHRL
[尝试次数：1] - [剩余数量：340] - [当前时间：20:49:41]

[ok] - 59252;GHRL
[尝试次数：2] - [剩余数量：339] - [当前时间：20:49:41]

[ok] - 621148F;GHRL
[尝试次数：1] - [剩余数量：338] - [当前时间：20:49:42]

[ok] - 621138F;GHRL
[尝试次数：1] - [剩余数量：337] - [当前时间：20:49:42]

[ok] - 621153F;GHRL
[尝试次数：1] - [剩余数量：336] - [当前时间：20:49:42]

[ok] - 621170F;GHRL
[尝试次数：1] - [剩余数量：335] - [当前时间：20:49:42]

[ok] - 621149F;GHRL
[尝试次数：1] - [剩余数量：334] - [当前时间：20:49:43]

[ok] - 59085;GHRL
[尝试次数：2] - [剩余数量：333] - [当前时间：20:49:43]

[ok] - 621181F;GHRL
[尝试次数：1] - [剩余数量：332] - [当前时间：20:49:43]

[ok] - 59085;GHRL
[尝试次数：2] - [剩余数量：331] - [当前时间：20:49:44]

[ok] - 621182F;GHRL
[尝试次数：1] -

[ok] - 221084M;GGNG
[尝试次数：3] - [剩余数量：210] - [当前时间：20:50:09]

[ok] - 221075M;GGNG
[尝试次数：2] - [剩余数量：209] - [当前时间：20:50:10]

[ok] - 221073M;GGNG
[尝试次数：3] - [剩余数量：208] - [当前时间：20:50:10]

[ok] - 221069M;GGNG
[尝试次数：4] - [剩余数量：207] - [当前时间：20:50:10]

[ok] - 221115M;GGNG
[尝试次数：1] - [剩余数量：206] - [当前时间：20:50:10]

[ok] - 221091M;GGNG
[尝试次数：5] - [剩余数量：205] - [当前时间：20:50:10]

[ok] - 221071M;GGNG
[尝试次数：2] - [剩余数量：204] - [当前时间：20:50:11]

[ok] - 221126M;GGNG
[尝试次数：1] - [剩余数量：203] - [当前时间：20:50:11]

[ok] - 221023M;GGNG
[尝试次数：6] - [剩余数量：202] - [当前时间：20:50:12]

[ok] - 221127M;GGNG
[尝试次数：1] - [剩余数量：201] - [当前时间：20:50:12]

[ok] - 221123M;GGNG
[尝试次数：3] - [剩余数量：200] - [当前时间：20:50:13]

[ok] - 221125M;GGNG
[尝试次数：2] - [剩余数量：199] - [当前时间：20:50:13]

[ok] - 221121M;GGNG
[尝试次数：4] - [剩余数量：198] - [当前时间：20:50:13]

[ok] - 221122M;GGNG
[尝试次数：1] - [剩余数量：197] - [当前时间：20:50:13]

[ok] - 221113M;GGNG
[尝试次数：4] - [剩余数量：196] - [当前时间：20:50:13]

[ok] - 221151;GGNG
[尝试次数：1] - [剩余数量：195] - [当前时间：20:50:14]

[ok] - 221364;GGNG
[尝试次数：

[ok] - 621364A;GHRL
[尝试次数：1] - [剩余数量：73] - [当前时间：20:50:57]

[ok] - 60892F;GHRL
[尝试次数：1] - [剩余数量：72] - [当前时间：20:50:58]

[ok] - 621364A;GHRL
[尝试次数：1] - [剩余数量：71] - [当前时间：20:50:58]

[ok] - 220101M;GGNG
[尝试次数：2] - [剩余数量：70] - [当前时间：20:50:58]

[ok] - 60897F;GHRL
[尝试次数：1] - [剩余数量：69] - [当前时间：20:50:58]

[ok] - 621364A;GHRL
[尝试次数：1] - [剩余数量：68] - [当前时间：20:50:59]

[ok] - 28411M;GGNG
[尝试次数：1] - [剩余数量：67] - [当前时间：20:51:00]

[ok] - 20864M;GGNG
[尝试次数：1] - [剩余数量：66] - [当前时间：20:51:00]

[ok] - 621364A;GHRL
[尝试次数：1] - [剩余数量：65] - [当前时间：20:51:00]

[ok] - 20864M;GGNG
[尝试次数：1] - [剩余数量：64] - [当前时间：20:51:00]

[ok] - MF21332N;GHRL
[尝试次数：1] - [剩余数量：63] - [当前时间：20:51:01]

[ok] - 20901M;GGNG
[尝试次数：1] - [剩余数量：62] - [当前时间：20:51:01]

[ok] - 20912M;GGNG
[尝试次数：1] - [剩余数量：61] - [当前时间：20:51:01]

[ok] - 220096M;GGNG
[尝试次数：1] - [剩余数量：60] - [当前时间：20:51:01]

[ok] - 621364A;GHRL
[尝试次数：2] - [剩余数量：59] - [当前时间：20:51:01]

[ok] - 20928M;GGNG
[尝试次数：1] - [剩余数量：58] - [当前时间：20:51:01]

[ok] - 221039M;GGNG
[尝试次数：1] - [剩余数量：57] - [当前时